In [1]:
import numpy as np

# Generate data

In [20]:
x = np.array([14,86,28,51,28,29,72,62,84,15,
              42,62,47,35,9,38,44,99,13,21,
              28,20,8,64,99,70,27,17,8])

B = 30
W = 2

y = B + W*x

# Stochastic Gradient Descent

Loss function is square error:

$$loss=(b+wx-y)^2$$

Gradients by chain rule:

$$F'(g(x)) = f’(g(x))g’(x)$$

$$\frac{dl}{db} = 2(b+wx-y)$$

$$\frac{dl}{dw} = 2(b+wx-y)*x$$

In [116]:
lr = 0.0001
epochs = 50

# Initial values
b = 0
w = 0

In [117]:
for e in range(epochs):
    for i in range(len(x)):
        
        # Gradient wrt to each parameter
        y_pred = b + w*x[i]
        dl_db = 2*(y_pred-y[i])
        dl_dw = 2*(y_pred-y[i])*x[i]
        
        # Update parameters
        b -= lr * dl_db
        w -= lr * dl_dw
        
print("After %d epochs:" % epochs)
print("b = %.2f (actual %.2f)" % (b,B))
print("w = %.2f (actual %.2f)" % (w,W))

After 50 epochs:
b = 2.83 (actual 30.00)
w = 2.58 (actual 2.00)


# Momentum

- Dampens oscillations by maintaining momentum from previous direction so steps don't zig zag as much, which should help get to minima faster
- Beta is commonly set to 0.9, so the parameter update is 90% same direction as previous step and 10% current gradient
- Note that previous step includes effect of all prior steps where the most recent ones are exponentially weighted (exponential average)


In [122]:
beta = 0.9
lr = 0.0001
epochs = 50

# Initial values
b = 0
w = 0
b_step_prev = 0
w_step_prev = 0

In [123]:
for e in range(epochs):
    for i in range(len(x)):
        
        # Gradient wrt to each parameter
        y_pred = b + w*x[i]
        dl_db = 2*(y_pred-y[i])
        dl_dw = 2*(y_pred-y[i])*x[i]

        # Step size for each parameter
        b_step = beta*b_step_prev + (1-beta)*dl_db
        w_step = beta*w_step_prev + (1-beta)*dl_dw
        b_step_prev = b_step
        w_step_prev = w_step
        
        # Update parameters
        b -= lr * b_step
        w -= lr * w_step

        
print("After %d epochs:" % epochs)
print("b = %.2f (actual %.2f)" % (b,B))
print("w = %.2f (actual %.2f)" % (w,W))

After 50 epochs:
b = 2.55 (actual 30.00)
w = 2.32 (actual 2.00)


# RMSprop

- Root Mean Square Propogation
- Dampens oscillations by taking smaller steps when previous steps have been large, and larger steps when previous steps have been small
    - Reduce step component perpendicular to direction of minima and increase step component directly towards it

In [144]:
beta = 0.9
lr = 0.01
epochs = 50

# Initial values
b = 0
w = 0
b_step_prev = 0
w_step_prev = 0

In [145]:
for e in range(epochs):
    for i in range(len(x)):
        
        # Gradient wrt to each parameter
        y_pred = b + w*x[i]
        dl_db = 2*(y_pred-y[i])
        dl_dw = 2*(y_pred-y[i])*x[i]

        # Step size for each parameter
        b_step = beta*b_step_prev + (1-beta)*dl_db**2
        w_step = beta*w_step_prev + (1-beta)*dl_dw**2
        b_step_prev = b_step
        w_step_prev = w_step
                
        # Update parameters
        b -= lr * dl_db/(b_step**0.5)
        w -= lr * dl_dw/(w_step**0.5)
        
        
print("After %d epochs:" % epochs)
print("b = %.2f (actual %.2f)" % (b,B))
print("w = %.2f (actual %.2f)" % (w,W))

After 50 epochs:
b = 8.53 (actual 30.00)
w = 2.37 (actual 2.00)


# Adam

- Adaptive Moment
- Combination of both Momentum and RMSProp

In [155]:
beta_m = 0.9
beta_r = 0.999
lr = 1.0
epochs = 50

# Initial values
b = 0
w = 0
b_m_step_prev = 0
w_m_step_prev = 0
b_r_step_prev = 0
w_r_step_prev = 0

In [161]:
for e in range(epochs):
    for i in range(len(x)):

        # Gradient wrt to each parameter
        y_pred = b + w*x[i]
        dl_db = 2*(y_pred-y[i])
        dl_dw = 2*(y_pred-y[i])*x[i]

        # Momentum step size for each parameter
        b_m_step = beta_m*b_m_step_prev + (1-beta_m)*dl_db
        w_m_step = beta_m*w_m_step_prev + (1-beta_m)*dl_dw
        b_m_step_prev = b_m_step
        w_m_step_prev = w_m_step

        # RMSProp step size for each parameter
        b_r_step = beta_r*b_r_step_prev + (1-beta_r)*dl_db**2
        w_r_step = beta_r*w_r_step_prev + (1-beta_r)*dl_dw**2
        b_r_step_prev = b_r_step
        w_r_step_prev = w_r_step

        # Update parameters
        b -= lr * b_m_step/(b_r_step**0.5)
        w -= lr * w_m_step/(w_r_step**0.5)
        
print("After %d epochs:" % epochs)
print("b = %.5f (actual %.2f)" % (b,B))
print("w = %.5f (actual %.2f)" % (w,W))

After 50 epochs:
b = 29.97153 (actual 30.00)
w = 2.04442 (actual 2.00)


#### Learning rate annealing

If we train for additional epochs, the parameters values will bounce around the true values. So even though Adam adjusts the step size, applying learning rate annealing (reducing) can help convergence.

#### Epsilon

Above implementation is missing the $\epsilon$ term in the parameter update:

$$w := w - \alpha \frac{\theta}{\sqrt{s}+\epsilon}$$

- $\alpha$ is the learning rate
- $\theta$ is the gradient

$\epsilon$ is suggested to be 1e-8 and doesn't usually require tuning

#### Bias correction

For exponentially weighted averages, the first few values won't have much history. So the This can be corrected by:

$$s_t = \beta s_{t-1} + (1-\beta) \theta_t$$

$$s_t^{corrected} = \frac{s_t}{1-\beta^t}$$

Practically speaking, because we usually train for many iterations, there probably isn't a need to apply this correction since it only really affects the first few steps.

Reference:
- [fast.ai Lesson 5](https://course.fast.ai/videos/?lesson=5)
- [Deeplearning.ai RMSprop](https://www.youtube.com/watch?v=_e-LFe_igno)
- [Deeplearning.ai Bias correction in exponentially weighted average](https://www.youtube.com/watch?v=lWzo8CajF5s)